# Common Time Series Models: Conditional Heteroskedasticity

In [ ]:
# !pip install arch

In [ ]:
import arch
import matplotlib.pyplot as plt
import numba
import numpy as np
import pandas as pd

from pandas_datareader import DataReader
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.arima.model import ARIMA

%matplotlib inline

## NASDAQ Returns

Recall our daily NASDAQ stock price data

In [ ]:
nasdaq = DataReader("NASDAQCOM", "fred", 2000, 2021).dropna()

In [ ]:
nasdaq.plot(figsize=(8, 6))

We would like to convert it into returns

In [ ]:
returns = np.log(nasdaq).diff().dropna()

In [ ]:
returns.plot()

### Heteroskedasticity

We differenced the data previously and noticed that the mean seemed to be close to constant, but that the variance seemed to vary which made the data non-stationary.

Is there a way to account for this?

Yes! Or we wouldn't be discussing it!

## ARCH: Autoregressive conditional heteroskedasticity

Let $\{Y_t\}$ follow some AR(p) process but we now allow for the variance to be time-varying:

\begin{align*}
  Y_{t+1} &= \mu + \sum_j (\rho_j Y_{t-j}) + \sigma_t \varepsilon_{t+1} \\
\end{align*}

We specify the process for $\sigma_t$ to be

\begin{align*}
  \sigma^2_t = \alpha_0 + \sum_{j=1}^q \alpha_j \varepsilon_{t-j}
\end{align*}

with $\alpha_j > 0$

In [ ]:
@numba.jit(nopython=True)
def simulate_ARCH(T, mu, rho, alphas):
    qSig = alphas.shape[0]

    Yt = np.zeros(T)
    sigma2t = np.zeros(T) + alphas[0]
    epsilont = np.random.randn(T + qSig)
    for t in range(1, T):
        # Just going to stick with AR(1) for now...
        Yt[t] = (1 - rho)*mu + rho*Yt[t-1] + np.sqrt(sigma2t[t-1])*epsilont[qSig+t]

        value = 0.0
        for j in range(qSig):
            value += alphas[1+j]*epsilont[qSig+t-j-1]
        sigma2t[t] = alphas[0] + value
    
    return Yt, epsilont[qSig:], sigma2t

In [ ]:
alphas = np.hstack([
    [1.5], [0.15*0.95**j for j in range(20)]
])
Yt, eps, sigma2t = simulate_ARCH(5000, 0.0, 0.0, alphas)

fig, ax = plt.subplots(2, figsize=(8, 6))

ax[0].plot(Yt)
ax[1].plot(sigma2t)

## GARCH: Generalized autoregressive conditional heteroskedasticity

We continue to let $\{Y_{t+1}\}$ to follow some AR(p) process.

\begin{align*}
  Y_{t+1} &= \mu + \sum_j (\rho_j Y_{t-j}) + \sigma_t \varepsilon_{t+1} \\
\end{align*}

but now we specify the process for $\sigma_t$ to be an ARMA(p, q) process

\begin{align*}
  \sigma_t = \alpha_0 + \sum_{j=1}^q \alpha_j \varepsilon_{t-j} + \sum_{j=1}^p \beta_j \sigma_{t-j}
\end{align*}

with $\alpha_j > 0$

**Beware**: Some packages use different $p$ and $q$ notation than we have -- Read documentation to ensure that they don't mean the opposite of what we've done.

### Fitting NASDAQ data


Unscaled returns

In [ ]:
mu = nasdaq["NASDAQCOM"].mean()
arch_model = arch.arch_model(returns["NASDAQCOM"] - mu)
res = arch_model.fit()
print(res.summary())

In [ ]:
(returns - mu).plot()

In [ ]:
res.plot();

Scaling the returns

In [ ]:
mu = nasdaq["NASDAQCOM"].mean()
arch_model = arch.arch_model(100*(returns["NASDAQCOM"] - mu))
res = arch_model.fit()
print(res.summary())

In [ ]:
res.plot();

Varying the parameters

In [ ]:
mu = nasdaq["NASDAQCOM"].mean()
# Scaled - Note that p corresponds to MA terms and q corresponds to the AR terms
arch_model = arch.arch_model(100*(returns["NASDAQCOM"] - mu), p=1, q=1)
res = arch_model.fit()
print(res.summary())

In [ ]:
res.plot();